In [1]:
import pandas as pd
import pyprind
import os
pbar = pyprind.ProgBar(50000)
labels={"pos":1, "neg":0}
df = pd.DataFrame()
for s in ("test", "train"):
    for l in ("pos", "neg"):
        path = "aclImdb/%s/%s" %(s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file)) as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
df.columns=["review", "sentiment"]

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:07:20


In [12]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv("movie_data.csv", index=False)
df = pd.read_csv("movie_data.csv")
df.columns=["review", "sentiment"]
df.head()

,review,sentiment
0,I really love this movie!!! I haven't played F...,1
1,I don't know what some people were thinking wh...,1
2,Moving beyond words is this heart breaking sto...,1
3,This is an excellent Anderson production worth...,1
4,"I thought the movie was good, but I like to re...",1


In [14]:
import pandas as pd
df = pd.read_csv("movie_data.csv")
df.columns=["review", "sentiment"]
df.head()

,review,sentiment
0,I really love this movie!!! I haven't played F...,1
1,I don't know what some people were thinking wh...,1
2,Moving beyond words is this heart breaking sto...,1
3,This is an excellent Anderson production worth...,1
4,"I thought the movie was good, but I like to re...",1


# tf - idf

In [15]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
'The sun is shining',
'The weather is sweet',
'The sun is shining and the weather is sweet'
    ])
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())

{'and': 0, 'is': 1, 'sun': 3, 'weather': 6, 'the': 5, 'sweet': 4, 'shining': 2}
[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


* tf(t, d) = so lan suat hien token t trong van ban d
* df(d, t) = so document d co chua t
* $idf(t,d) = \log\frac{n_d}{1 + df(d,t)}$
* $tf-idf = tf(t,d) * idf(t,d)$

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[ 0.    0.43  0.56  0.56  0.    0.43  0.  ]
 [ 0.    0.43  0.    0.    0.56  0.43  0.56]
 [ 0.4   0.48  0.31  0.31  0.31  0.48  0.31]]


### in sklearn:
* $idf(t,d) = \log\frac{1 + n_d}{1 + df(d,t}$
* $tf-idf(t,d) = tf(t,d) * (idf(t,d)+1 ) $

# cleaning text data

In [17]:
import re
def preprocessor(text):
    text = re.sub(r"<[^>]>", '', text)
    emoticons = re.findall("(?:|;|=)(?:-)?(?:\)\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + \
            " ".join(emoticons).replace('-', '')
    return text
df["review"] = df["review"].apply(preprocessor)

## Processing documents into tokens

In [18]:
def tokenizer(text):
    return text.split()

In [19]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [20]:
from nltk.corpus import stopwords
stop = stopwords.words("english")
[w for w in tokenizer_porter("a runners like running and thus they run")
    if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

## Training a logistic regression model for document classification

In [21]:
X_train = df.loc[:25000, "review"].values
y_train = df.loc[:25000, "sentiment"].values
X_test = df.loc[25000:, "review"].values
y_test = df.loc[25000:, "sentiment"].values

In [22]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)
param_grid = [{"vect__ngram_range": [(1,1)],
               "vect__stop_words": [stop, None],
               "vect__tokenizer": [tokenizer,
                               tokenizer_porter],
               "clf__penalty": ["l1", "l2"],
               "clf__C": [1.0, 10.0, 100.0]},
              {"vect__ngram_range": [(1,1)],
               "vect__stop_words": [stop, None],
               "vect__tokenizer": [tokenizer,
                                 tokenizer_porter],
               "vect__use_idf":[False],
               "vect__norm":[None],
               "clf__penalty": ["l1", "l2"],
               "clf__C": [1.0, 10.0, 100.0]}
             ]
lr_tfidf = Pipeline([("vect", tfidf),
                    ("clf",
                     LogisticRegression(random_state=0))]
                   )
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                          scoring="accuracy",
                          cv=5, verbose=1,
                          n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/jeovach/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jeovach/anaconda3/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/jeovach/anaconda3/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/home/jeovach/anaconda3/lib/

In [ ]:
print(gs_lr_tfidf.best_params_)

In [23]:
print(gs_lr_tfidf.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [24]:
clf = gs_lr_tfidf.best_estimator_
clf.score(X_test, y_test)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [25]:
def tokenizer(text):
    text = re.sub(r"<[^>]>", '', text)
    emoticons = re.findall("(?:|;|=)(?:-)?(?:\)\(|D|P)", text)
    text = re.sub("[\W]+", " ", text.lower()) + \
            " ".join(emoticons).replace('-', '')
    tokenized = [w for w in text.split()
                if w not in stop]
    return tokenized

In [26]:
def stream_docs(path):
    with open(path, "r") as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [42]:
next(stream_docs("movie_data.csv"))

('"This is an excellent film, with an extraordinary cast and acting. I was very disappointed with the Academy Awards when this didn\'t get the Oscar for best film and for best actress (Woopi Goldberg)... it certainly deserved it. In any case, take a look at it. i am sure you will enjoy it very much."',
 1)

In [39]:
def get_minibatch(doc_stream, size):
    docs , y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [40]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error="ignore",
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)
print(vect)
clf = SGDClassifier(loss="log", random_state=1, n_iter=1)
doc_stream = stream_docs("movie_data.csv")

In [41]:
import pyprind
import numpy as np
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream,
                                    size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:12


In [43]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
clf.score(X_test, y_test)

0.87839999999999996